In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import sys

sys.path.append(os.path.join(os.environ['REPO_DIR'], 'utilities'))
from utilities2015 import *
from metadata import *
from data_manager import *
from distributed_utilities import *

Setting environment for AWS compute node


No vtk
File does not exist: /shared/CSHL_data_processed/MD585/MD585_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD585/MD585_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD589/MD589_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_anchor.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_sorted_filenames.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD590/MD590_cropbox.txt
File does not exist: /shared/CSHL_data_processed/MD591/MD591_anchor.txt
File does not exist: 

In [5]:
stack = 'MD589'

In [24]:
transfer_data_synced(os.path.join('CSHL_data', stack), 
                     from_hostname='s3', to_hostname='ec2', is_dir=True, include_only='*.tif')

rm -rf /shared/CSHL_data/MD589 && mkdir -p /shared/CSHL_data
aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data/MD589 /shared/CSHL_data/MD589 --include "*.tif"


Child returned 0
Child returned 0


In [6]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_sorted_filenames.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)

rm -rf /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_sorted_filenames.txt /shared/CSHL_data_processed/MD589/MD589_sorted_filenames.txt


Child returned 0
Child returned 0


In [8]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_elastix_output'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=True)

rm -rf /shared/CSHL_data_processed/MD589/MD589_elastix_output && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp --recursive s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_elastix_output /shared/CSHL_data_processed/MD589/MD589_elastix_output


Child returned 0
Child returned 0


# Align

In [7]:
_, sections_to_filenames = DataManager.load_sorted_filenames(stack=stack)
filenames = [fn for fn in sections_to_filenames.values() if not is_invalid(fn=fn)]

In [3]:
run_distributed("%(script)s %(stack)s %(input_dir)s %(elastix_output_dir)s \'%%(kwargs_str)s\'" % \
                {'stack': stack,
                'script': os.path.join(REPO_DIR, 'preprocess', 'align_consecutive_v2.py'),
                'input_dir': os.path.join(RAW_DATA_DIR, stack),
                'elastix_output_dir': os.path.join(DATA_DIR, stack, stack+'_elastix_output')},
                kwargs_list=[{'prev_fn': filenames[i-1], 'curr_fn': filenames[i]} for i in range(1, len(filenames))],
                argument_type='list',
               cluster_size=12,
                exclude_nodes=[33])

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


# Compose

In [9]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_anchor.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)

rm -rf /shared/CSHL_data_processed/MD589/MD589_anchor.txt && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_anchor.txt /shared/CSHL_data_processed/MD589/MD589_anchor.txt


Child returned 0
Child returned 0


In [10]:
anchor_fn = DataManager.load_anchor_filename(stack=stack)

In [11]:
elastix_output_dir = os.path.join(DATA_DIR, stack, stack + '_elastix_output')
output_fn = os.path.join(elastix_output_dir, '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                                dict(stack=stack, anchor_fn=anchor_fn))

In [6]:
! rm -f $output_fn

In [7]:
t = time.time()
print 'Composing transform...'

run_distributed("%(script)s %(stack)s %(elastix_output_dir)s \'%%(kwargs_str)s\' %(anchor_idx)d %(output_fn)s" % \
            {'stack': stack,
            'script': os.path.join(REPO_DIR, 'preprocess', 'compose_transform_thumbnail_v2.py'),
            'elastix_output_dir': elastix_output_dir,
            'anchor_idx': filenames.index(anchor_fn),
            'output_fn': output_fn},
            kwargs_list=[{'filenames': filenames}],
            argument_type='list',
               cluster_size=1,
               exclude_nodes=[33])

linked_name = os.path.join(elastix_output_dir, '%(stack)s_transformsTo_anchor.pkl' % dict(stack=stack))
execute_command('rm ' + linked_name)
execute_command('ln -s ' + output_fn + ' ' + linked_name)

print 'done in', time.time() - t, 'seconds'

Composing transform...
['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']
rm /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_elastix_output/MD657_transformsTo_anchor.pkl
ln -s /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_elastix_output/MD657_transformsTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130.pkl /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_elastix_output/MD657_transformsTo_anchor.pkl
done in 22.394327879 seconds


Child returned 0
Child returned 0


# Warp

In [5]:
tb_fmt = 'png'
pad_bg_color = 'auto'

In [9]:
input_dir = os.path.join(RAW_DATA_DIR, stack)
out_dir = os.path.join(DATA_DIR, stack, stack + '_thumbnail_alignedTo_' + anchor_fn)
script = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')

In [10]:
! rm -r $out_dir

In [11]:
t = time.time()
print 'Warping...'

transforms_filename = os.path.join(elastix_output_dir,
                    '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                    dict(stack=stack, anchor_fn=anchor_fn))
transforms_to_anchor = pickle.load(open(transforms_filename, 'r'))

if pad_bg_color == 'auto':
    run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                    {'script': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'out_dir': out_dir
                    },
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'filename': fn + '.' + tb_fmt,
                                'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif',
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in filenames],
                    exclude_nodes=[33],
                    argument_type='single',
                   cluster_size=12)
else:
    run_distributed('%(script)s %(stack)s %(input_dir)s %(out_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 %%(pad_bg_color)s' % \
                    {'script_dir': script,
                    'stack': stack,
                    'input_dir': input_dir,
                    'out_dir': out_dir,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                                'filename': fn + '.' + tb_fmt,
                                'output_fn': fn + '_thumbnail_alignedTo_' + anchor_fn + '.tif'}
                                for fn in filenames],
                    exclude_nodes=[33],
                    argument_type='single',
                   cluster_size=12)

print 'done in', time.time() - t, 'seconds'

Warping...
['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']
done in 631.269037008 seconds


In [47]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_thumbnails_alignedTo_' + anchor_fn),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

aws s3 cp --recursive /shared/CSHL_data_processed/MD589/MD589_thumbnails_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091 s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_thumbnails_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091


Child returned 0


# Crop

In [12]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_cropbox.txt'), 
                     from_hostname='s3', to_hostname='ec2', is_dir=False)

rm -rf /shared/CSHL_data_processed/MD589/MD589_cropbox.txt && mkdir -p /shared/CSHL_data_processed/MD589
aws s3 cp s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_cropbox.txt /shared/CSHL_data_processed/MD589/MD589_cropbox.txt


Child returned 0
Child returned 0


In [13]:
xmin, xmax, ymin, ymax, first_sec, last_sec = DataManager.load_cropbox(stack=stack)
w = xmax + 1 - xmin
h = ymax + 1 - ymin
x = xmin
y = ymin

In [14]:
first_fn = filenames[first_sec-1]
last_fn = filenames[last_sec-1]

In [15]:
first_idx = filenames.index(first_fn)
last_idx = filenames.index(last_fn)

# Crop Thumbnail

In [19]:
input_dir = os.path.join(DATA_DIR, stack, "%(stack)s_thumbnail_alignedTo_%(anchor_fn)s" % \
                           {'stack': stack, 'anchor_fn': anchor_fn})

output_dir = os.path.join(DATA_DIR, stack, "%(stack)s_thumbnail_alignedTo_%(anchor_fn)s_cropped" % \
                           {'stack': stack, 'anchor_fn': anchor_fn})

execute_command('mkdir -p ' + output_dir)

mkdir -p /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_thumbnail_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped


Child returned 0


0

In [20]:
t = time.time()
sys.stderr.write('cropping thumbnail...')

execute_command('mogrify -set filename:name %%t -crop %(w)dx%(h)d+%(x)d+%(y)d -write "%(output_dir)s/%%[filename:name]_cropped.tif" %(input_dir)s/*.tif' % \
    {'input_dir': input_dir,
     'output_dir': output_dir,
    'w':w, 'h':h, 'x':x, 'y':y})

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 100 seconds

cropping thumbnail...

mogrify -set filename:name %t -crop 862x530+774+247 -write "/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_thumbnail_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped/%[filename:name]_cropped.tif" /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_thumbnail_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130/*.tif


Child returned 0
done in 96.765667 seconds


In [75]:
transfer_data_synced(os.path.join('CSHL_data_processed', stack, stack + '_thumbnails_alignedTo_' + anchor_fn + '_cropped'),
                    from_hostname='ec2',
                    to_hostname='s3',
                    is_dir=True)

aws s3 cp --recursive /shared/CSHL_data_processed/MD589/MD589_thumbnails_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped s3://mousebrainatlas-data/CSHL_data_processed/MD589/MD589_thumbnails_alignedTo_MD589-IHC31-2015.07.30-23.26.22_MD589_1_0091_cropped


Child returned 0


# Expand lossless JP2

In [18]:
transfer_data_synced(os.path.join('CSHL_data', stack),
                    from_hostname='s3raw',
                    to_hostname='ec2',
                    is_dir=True,
                    include_only='*_lossless.jp2')

aws s3 cp --recursive s3://mousebrainatlas-rawdata/CSHL_data/MD589 /shared/CSHL_data/MD589 --include "*_lossless.jp2"


KeyboardInterrupt: 

In [23]:
t = time.time()
sys.stderr.write('expanding...')

input_dir = create_if_not_exists(os.path.join(DATA_DIR, stack, stack + '_lossless_tif'))
output_dir = os.path.join(RAW_DATA_DIR, stack)

filenames_to_expand = [fn for fn in filenames[first_idx:last_idx+1] if not os.path.exists(os.path.join(input_dir, fn + '_lossless.tif'))]
# sys.stderr.write('filenames_to_expand: %s' % filenames_to_expand)

run_distributed('kdu_expand_patched -i %(output_dir)s/%%(fn)s_lossless.jp2 -o %(input_dir)s/%%(fn)s_lossless.tif' % \
                {'output_dir': output_dir,
                'input_dir': input_dir},
                kwargs_list={'fn': filenames_to_expand},
                exclude_nodes=[33],
                argument_type='single',
               cluster_size=12)

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 900 seconds.

expanding...filenames_to_expand: ['MD657-N13-2017.02.22-14.33.34_MD657_3_0039', 'MD657-F13-2017.02.17-20.12.13_MD657_3_0039', 'MD657-N14-2017.02.22-14.37.44_MD657_1_0040', 'MD657-F14-2017.02.17-20.29.47_MD657_1_0040', 'MD657-N14-2017.02.22-14.37.44_MD657_2_0041', 'MD657-F14-2017.02.17-20.29.47_MD657_2_0041', 'MD657-N14-2017.02.22-14.37.44_MD657_3_0042', 'MD657-F14-2017.02.17-20.29.47_MD657_3_0042', 'MD657-N15-2017.02.22-14.41.51_MD657_1_0043', 'MD657-F15-2017.02.17-20.46.06_MD657_1_0043', 'MD657-N15-2017.02.22-14.41.51_MD657_2_0044', 'MD657-F15-2017.02.17-20.46.06_MD657_2_0044', 'MD657-N15-2017.02.22-14.41.51_MD657_3_0045', 'MD657-F15-2017.02.17-20.46.06_MD657_3_0045', 'MD657-N16-2017.02.27-16.09.46_MD657_1_0046', 'MD657-F16-2017.02.17-21.02.17_MD657_1_0046', 'MD657-N16-2017.02.27-16.09.46_MD657_2_0047', 'MD657-F16-2017.02.17-21.02.17_MD657_2_0047', 'MD657-N16-2017.02.27-16.09.46_MD657_3_0048', 'MD657-F16-2017.02.17-21.02.17_MD657_3_0048', 'MD657-N17-2017.02.22-14.45.59_MD657_1_0049', 

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 974.662535 seconds


# Warping and cropping lossless

In [8]:
tf_filepath = os.path.join(DATA_DIR, stack, stack+'_elastix_output', '%(stack)s_transformsTo_anchor.pkl' % {'stack':stack})
tfs = pickle.load(open(tf_filepath, 'r'))
# Note that the index from trasform pickle file starts at 0, BUT the .._renamed folder index starts at 1.#

lossless_tif_dir = os.path.join(DATA_DIR, stack, stack + '_lossless_tif')
lossless_aligned_cropped_dir = os.path.join(DATA_DIR, stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped')

script_fp = os.path.join(REPO_DIR, 'preprocess', 'warp_crop_IM_v2.py')

In [9]:
t = time.time()
sys.stderr.write('warping and cropping lossless...')
                   
if pad_bg_color == 'auto':
    # If alternating, then black padding for F sections, white padding for N sections.
    run_distributed(command='%(script_path)s %(stack)s %(lossless_tif_dir)s %(lossless_aligned_cropped_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %%(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'lossless_tif_dir': lossless_tif_dir,
                    'lossless_aligned_cropped_dir': lossless_aligned_cropped_dir,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h},
                    kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                'filename': fn + '_lossless.tif',
                                'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif',
                                'pad_bg_color': 'black' if fn.split('-')[1][0] == 'F' else 'white'}
                                for fn in filenames[first_idx:last_idx+1]],
                    exclude_nodes=[33],
                    argument_type='single')
else:
    run_distributed(command='%(script_path)s %(stack)s %(lossless_tif_dir)s %(lossless_aligned_cropped_dir)s %%(transform)s %%(filename)s %%(output_fn)s lossless %(x)d %(y)d %(w)d %(h)d %(pad_bg_color)s'%\
                    {'script_path': script_fp,
                    'stack': stack,
                    'lossless_tif_dir': lossless_tif_dir,
                    'lossless_aligned_cropped_dir': lossless_aligned_cropped_dir,
                    'x': x,
                    'y': y,
                    'w': w,
                    'h': h,
                    'pad_bg_color': pad_bg_color},
                    kwargs_list=[{'transform': ','.join(map(str, tfs[fn].flatten())),
                                'filename': fn + '_lossless.tif',
                                'output_fn': fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif'}
                                for fn in filenames[first_idx:last_idx+1]],
                    exclude_nodes=[33],
                    argument_type='single')

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 500 seconds.

warping and cropping lossless...

['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']


done in 471.258449 seconds


# Generate other versions

In [13]:
script_fp = os.path.join(REPO_DIR, 'preprocess', 'generate_other_versions_v2.py')
input_dir = os.path.join(data_dir, stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped')
out_sat_dir = os.path.join(data_dir, stack, stack + '_lossless_alignedTo_' + anchor_fn + '_cropped_saturation')

In [ ]:
# t = time.time()
# print 'Generating saturation image...',

# run_distributed4('%(script)s %(stack)s %(input_dir)s \'%%(input_filenames)s\' --output_saturation_dir %(output_saturation_dir)s' % \
#                 dict(script=script_fp,
#                      stack=stack,
#                      input_dir=input_dir,
#                      output_saturation_dir=out_sat_dir,
#                      kwargs_list={'input_filenames': [fn + '_lossless_alignedTo_' + anchor_fn + '_cropped.tif' for fn in filenames[first_idx:last_idx+1]]},
#                     exclude_nodes=exclude_nodes,
#                     argument_type='list2')

# print 'done in', time.time() - t, 'seconds'

# Generate Masks

In [4]:
from preprocess_utilities import *

In [5]:
input_dir = os.path.join(RAW_DATA_DIR, stack)
output_dir = create_if_not_exists(os.path.join(DATA_DIR, stack, stack + '_submasks'))

In [6]:
border_dissim_percentile = DEFAULT_BORDER_DISSIMILARITY_PERCENTILE
min_size = DEFAULT_MINSIZE

In [7]:
t = time.time()
print 'Generating thumbnail mask...',

# !! For some reason (perhaps too much simultaneous write to disk), the distributed computation cannot finish, usually stuck with only a few sections left.
script = os.path.join(os.environ['REPO_DIR'], 'preprocess', 'generate_thumbnail_masks_v4.py')

run_distributed(command='%(script_path)s %(stack)s %(input_dir)s \'%%(filenames)s\' %(output_dir)s --border_dissim_percentile %(border_dissim_percentile)d --min_size %(min_size)d' % \
                {'script_path': script,
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir,
                'border_dissim_percentile': border_dissim_percentile,
                'min_size': min_size},
                kwargs_list=dict(filenames=filenames),
                exclude_nodes=[33],
                argument_type='list2')

print 'done in', time.time() - t, 'seconds' # 1600s

Generating thumbnail mask... ['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']
done in 551.316730022 seconds


In [ ]:
# ... Review in local machine using GUI...

# Warp Thumbnail Masks

In [11]:
script = os.path.join(os.environ['REPO_DIR'], 'preprocess', 'warp_crop_IM_v2.py')
input_dir = os.path.join(DATA_DIR, stack, stack + '_masks')
output_dir = os.path.join(DATA_DIR, stack, stack + '_masks_alignedTo_' + anchor_fn)

execute_command('rm -rf ' + output_dir)

transforms_filename = os.path.join(DATA_DIR, stack, stack+'_elastix_output', 
                                   '%(stack)s_transformsTo_%(anchor_fn)s.pkl' % \
                                   dict(stack=stack, anchor_fn=anchor_fn))
transforms_to_anchor = pickle.load(open(transforms_filename, 'r'))

rm -rf /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_masks_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130


Child returned 0


In [12]:
t = time.time()
print 'warping thumbnail mask...',

run_distributed4('%(script)s %(stack)s %(input_dir)s %(output_dir)s %%(transform)s %%(filename)s %%(output_fn)s thumbnail 0 0 2000 1500 black' % \
                {'script': script,
                'stack': stack,
                'input_dir': input_dir,
                'output_dir': output_dir},
                kwargs_list=[{'transform': ','.join(map(str, transforms_to_anchor[fn].flatten())),
                            'filename': fn + '_mask.png',
                            'output_fn': fn + '_mask_alignedTo_' + anchor_fn + '.png'}
                            for fn in filenames],
                exclude_nodes=[33],
                argument_type='single')

print 'done in', time.time() - t, 'seconds' # 780

warping thumbnail mask... ['gcn-20-33.sdsc.edu'] are excluded
Using nodes: ['gcn-20-31.sdsc.edu', 'gcn-20-32.sdsc.edu', 'gcn-20-34.sdsc.edu', 'gcn-20-35.sdsc.edu', 'gcn-20-36.sdsc.edu', 'gcn-20-37.sdsc.edu', 'gcn-20-38.sdsc.edu', 'gcn-20-41.sdsc.edu', 'gcn-20-42.sdsc.edu', 'gcn-20-43.sdsc.edu', 'gcn-20-44.sdsc.edu', 'gcn-20-45.sdsc.edu', 'gcn-20-46.sdsc.edu', 'gcn-20-47.sdsc.edu', 'gcn-20-48.sdsc.edu']
done in 780.778076887 seconds


# Crop Thumbnail Masks

In [13]:
input_dir = os.path.join(DATA_DIR, stack, stack + '_masks_alignedTo_' + anchor_fn)
output_dir = os.path.join(DATA_DIR, stack, stack + '_masks_alignedTo_' + anchor_fn + '_cropped')

execute_command('rm -rf ' + output_dir)
execute_command('mkdir -p ' + output_dir)

rm -rf /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_masks_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped
mkdir -p /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_masks_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped


Child returned 0
Child returned 0


0

In [14]:
t = time.time()
sys.stderr.write('cropping thumbnail mask...')

execute_command('mogrify -set filename:name %%t -crop %(w)dx%(h)d+%(x)d+%(y)d -write "%(output_dir)s/%%[filename:name]_cropped.png" %(input_dir)s/*.png' % \
    {'stack': stack,
    'input_dir': input_dir,
    'output_dir': output_dir,
    'w':xmax+1-xmin, 'h':ymax+1-ymin, 'x':xmin, 'y':ymin})

sys.stderr.write('done in %f seconds\n' % (time.time() - t)) # 70s

cropping thumbnail mask...

mogrify -set filename:name %t -crop 862x530+774+247 -write "/oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_masks_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped/%[filename:name]_cropped.png" /oasis/projects/nsf/csd395/yuncong/CSHL_data_processed/MD657/MD657_masks_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130/*.png


Child returned 0
done in 73.238121 seconds


In [12]:
! tail /tmp/stderr.log

Output image already exists. MD657-N70-2017.02.22-18.31.14_MD657_2_0209_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped.tif
Output image already exists. MD657-N51-2017.02.22-16.54.20_MD657_2_0152_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped.tif
Output image already exists. MD657-F37-2017.02.18-03.39.28_MD657_1_0109_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped.tif
Output image already exists. MD657-N79-2017.02.22-19.01.38_MD657_2_0236_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped.tif
Output image already exists. MD657-F32-2017.02.18-01.56.25_MD657_2_0095_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped.tif
Output image already exists. MD657-F16-2017.02.17-21.02.17_MD657_2_0047_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD657_1_0130_cropped.tif
Output image already exists. MD657-N37-2017.02.22-15.55.56_MD657_2_0110_lossless_alignedTo_MD657-F44-2017.02.18-06.06.27_MD6